In [1]:
import pandas as pd
import numpy as np
import time
import rrBLUP as p_rrBlup
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula as f
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro
pandas2ri.activate()
rrBLUP = importr('rrBLUP')
df_geno = pd.read_csv('data/23TC1_YLD14_SU_geno_final.txt', sep='\t')
df_geno = df_geno.sort_values(by=['Genotype'])
print(df_geno.info())
df_pheno = pd.read_csv("data/23TC1_YLD14_SU_pheno_final.txt", sep='\t')


<class 'pandas.core.frame.DataFrame'>
Index: 2763 entries, 112 to 2396
Columns: 9642 entries, Genotype to 10_149535062_G
dtypes: int64(9641), object(1)
memory usage: 203.3+ MB
None


In [2]:

df_pheno_ordered = df_pheno.sort_values(by=['Genotype'])
print(df_pheno_ordered.info())
train_x = np.array(df_geno.drop('Genotype', axis=1))
train_y = np.array(df_pheno_ordered['YLD14_SU']).reshape(-1, 1)
train_x = train_x - 1
print(train_x.shape)
print(train_y.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 2763 entries, 1929 to 179
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Genotype  2763 non-null   object 
 1   YLD14_SU  2763 non-null   float64
dtypes: float64(1), object(1)
memory usage: 64.8+ KB
None
(2763, 9641)
(2763, 1)


In [3]:
a_1 = rrBLUP.A_mat(train_x)
a_1

array([[ 0.96460593,  0.40092433,  0.25768902, ..., -0.30201703,
        -0.23842051, -0.23847136],
       [ 0.40092433,  0.81633067,  0.42894227, ..., -0.22019376,
        -0.27356791, -0.27299987],
       [ 0.25768902,  0.42894227,  0.75699359, ..., -0.24150724,
        -0.2534156 , -0.25408534],
       ...,
       [-0.30201703, -0.22019376, -0.24150724, ...,  0.70114823,
         0.31388157,  0.31383073],
       [-0.23842051, -0.27356791, -0.2534156 , ...,  0.31388157,
         0.89487222,  0.89420248],
       [-0.23847136, -0.27299987, -0.25408534, ...,  0.31383073,
         0.89420248,  0.89569887]])

In [52]:
type(a_1)

numpy.ndarray

In [5]:
import aMat
a_2 = aMat.add_mat_numpy(train_x+1)

In [53]:
import a_mat_jax
import jax
a_3 = a_mat_jax.add_mat_jax(train_x+1)
a_3 = np.asarray(a_3)
type(a_3)

numpy.ndarray

In [54]:
# Using allclose to compare
close = np.allclose(a_2, a_3, rtol=0.00001, atol=0.00001)

print(f"Arrays are close: {close}")

Arrays are close: True


In [4]:
start_time = time.time()
g_result = rrBLUP.mixed_solve(y = train_y, K = a_1)
end_time = time.time()
#print(g_result)
print('Running Time: '+str(round(end_time-start_time, 2))+' s')
#train_pred = g_result['u'] 
train_pred = np.array(g_result.rx2('u'))

Running Time: 20.03 s


AttributeError: 'dict' object has no attribute 'rx2'

In [39]:
train_y

array([[689.5701105],
       [615.9386355],
       [661.3197776],
       ...,
       [692.8649345],
       [709.3628695],
       [684.4678623]])

In [64]:
start_time = time.time()
g_result = p_rrBlup.mixed_solve(y = train_y, K = a_2)
end_time = time.time()
print(g_result)
print('Running Time: '+str(round(end_time-start_time, 2))+' s')
#train_pred = g_result['u'] 
rrblup_u = g_result['u']

{'Vu': 462.6130366195777, 'Ve': 969.4607183018168, 'beta': array([[703.72274814]]), 'u': array([[-17.58200532],
       [-21.2687656 ],
       [ -0.33706766],
       ...,
       [ -4.68147966],
       [-19.89370123],
       [-20.12403582]]), 'LL': -13572.19170915865}
Running Time: 17.25 s


In [7]:
rrblup_u

array([[-17.59836984],
       [-20.95136378],
       [ -0.13633498],
       ...,
       [ -4.65200707],
       [-20.01554518],
       [-20.12818341]])

In [76]:
import gblup
g_result = gblup.solve_mixed_gblup_model(n_obs=len(train_y), obs_ids=[i for i in range(len(train_y))], y_values=train_y, ainv=np.linalg.inv(a_2))
gblue_values = g_result[1:,]
from scipy.stats import pearsonr

rrblup_u_1d = rrblup_u.flatten()
gblue_values_1d = gblue_values.flatten()

# Now you can calculate the correlation coefficient
correlation_coef = np.corrcoef(rrblup_u_1d, gblue_values_1d)[0, 1]
print("Correlation Coefficient (相关系数):", correlation_coef)


Correlation Coefficient (相关系数): 0.9999505295951803


In [8]:
asreml = importr('asreml')

Online License checked out Mon Mar 25 13:40:58 2024


In [55]:
from rpy2.robjects import r, pandas2ri
# Transfer G_matrix to R
r.assign('ID', df_pheno_ordered['Genotype'])
r.assign('Gmat', a_2)
r('''
# Adjust G_matrix and invert
library(ASRgenomics)
rownames(Gmat) = ID
colnames(Gmat) = ID
diag(Gmat) <- diag(Gmat) + 0.01
ginv = G.inverse(Gmat,sparseform = T)$Ginv.sparse''')


R[write to console]: Reciprocal conditional number for original matrix is: 1.59179018378963e-06

R[write to console]: Reciprocal conditional number for inverted matrix is: 1.24079893155397e-06

R[write to console]: Inverse of matrix G does not appear to be ill-conditioned.



array([[ 1.00000000e+00,  1.00000000e+00,  1.04168312e+01],
       [ 2.00000000e+00,  1.00000000e+00,  5.02454510e-01],
       [ 2.00000000e+00,  2.00000000e+00,  2.62023822e+01],
       ...,
       [ 2.76300000e+03,  2.76100000e+03, -2.16311000e-03],
       [ 2.76300000e+03,  2.76200000e+03, -2.17669057e+01],
       [ 2.76300000e+03,  2.76300000e+03,  2.63256497e+01]])

In [18]:

df_pheno_ordered['Genotype'] = df_pheno_ordered['Genotype'].astype('category')

In [56]:
import rpy2.robjects as robjects

modg = asreml.asreml(
fixed = f('YLD14_SU ~ 1'),
random = f('~vm(Genotype,ginv)'),
dense = f('~vm(Genotype,ginv)'),
residual =f('~idv(units)'), 
workspace = "2Gb",
data = df_pheno_ordered)

In [21]:
summ = asreml.summary_asreml(modg)
print(summ.rx2('varcomp'))

                              component std.error  z.ratio bound %ch
vm(Genotype, G_matrix_inv) 5.972789e-06        NA       NA     B  NA
units!units                1.170023e+03  31.48448 37.16189     P   0
units!R                    1.000000e+00        NA       NA     F   0


In [58]:
asreml_u = (modg.rx2['coefficients'].rx2['random'])


In [63]:
close = np.allclose(rrblup_u, asreml_u, rtol=0.00001, atol=0.00001)



array([[-17.5657198 ],
       [-21.58628211],
       [ -0.53769809],
       ...,
       [ -4.71096706],
       [-19.77202265],
       [-20.12006455]])

In [70]:
from scipy.stats import pearsonr

rrblup_u_1d = rrblup_u.flatten()
asreml_u_1d = asreml_u.flatten()

# Now you can calculate the correlation coefficient
correlation_coef = np.corrcoef(rrblup_u_1d, asreml_u_1d)[0, 1]
print("Correlation Coefficient (相关系数):", correlation_coef)


Correlation Coefficient (相关系数): 0.9999217812834612
